In [1]:
import ipywidgets
import scipy.linalg as lin
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# %load cheb.py
from numpy import *
from numpy.fft import fft,ifft

def cheb(y):
    '''Chebyshev transform. Finds Chebyshev coefficients given y evaluated on
    Chebyshev grid'''
    N = len(y) - 1
    yt = real(fft(r_[y,y[-2:0:-1]]))
    yt = yt/(2*N)
    yt = r_[yt[0],yt[1:N]+yt[-1:N:-1],yt[N]]

    return yt

def icheb(c):
    '''Inverse Chebyshev transform. Evaluates Chebyshev series at the Chebyshev 
    grid points given Chebyshev coefficients.'''
    N = len(c) - 1
    y = r_[c[0],0.5*c[1:N],c[N],0.5*c[-1:N:-1]]
    y = y*2*N
    y = real(ifft(r_[y,y[-2:0:-1]]))[:N+1]

    return y


def Dcheb(y,interval):
    '''Chebyshev derivative of y evaluated on Chebyshev grid in interval [a,b]'''
    N = len(y) - 1
    a,b = interval
    x = 0.5*(b-a)*(cos(r_[0:N+1]*pi/N) + 1) + a

    k = r_[0:N]

    A = real(fft(r_[y,y[-2:0:-1]]))
    yy = real(ifft(1j*r_[k,0,-k[-1:0:-1]]*A))

    fact = 2.*(x-a)/(b-a)-1
    fact = fact[1:-1]
    yprime = -2./(b-a)*yy[1:N]/sqrt(1-fact**2)

    A = A/(2*N)
    A = r_[A[0],A[1:N]+A[-1:N:-1],A[N]]
    k = r_[0:N+1]

    yprime1 = sum(k**2*A)*2./(b-a)
    yprimeN = sum((-1)**(k+1)*k**2*A)*2./(b-a)

    return r_[yprime1,yprime,yprimeN]



def regrid(y,M):
    N = len(y) - 1
    a = cheb(y)
    if M==N:
        return y
    if M>N:
        a = r_[a,zeros(M-N)]
        return icheb(a)
    if M<N:
        a = a[:M+1]
        return icheb(a)



def clenshaw(x,c):
    '''Clenshaw algorithm to evaluate Chebyshev series at x
    assumes x is in [-1,1]'''
    N = len(c) - 1
    b = zeros(N+2)
    b[-1] = 0
    b[-2] = c[-1]
    for r in r_[N-1:0:-1]:
        b[r] = 2*x*b[r+1] - b[r+2] + c[r]
    s = x*b[1] - b[2] + c[0]

    return s


def clenshaw2(x,c,change_grid = True):
    '''Vectorized version of Clenshaw algorithm
    Use this for Chebyshev polynomial evaluation'''
    if change_grid:
    	if (min(x)!=-1) or (max(x)!=1):
        	x = 2*(x-min(x))/(max(x)-min(x)) - 1
    N = len(c) - 1
    b = zeros([N+2,len(x)])
    b[-1,:] = 0
    b[-2,:] = c[-1]
    for r in r_[N-1:0:-1]:
        b[r,:] = 2*x*b[r+1,:] - b[r+2,:] + c[r]
    s = x*b[1,:] - b[2,:] + c[0]

    return s



def chebD(c,interval):
    '''Finds derivative of Chebyshev series in spectral space
    i.e. maps c_n--->d_n where c_n,d_n are Chebyshev coefficients
    of f(x) and f'(x) in the interval [a,b].'''
    N = len(c) - 1
    a,b = interval
    if (a!=-1.) or (b!=1.):
        factor = 2./(b-a)
    else: 
        factor = 1.

    b = c*r_[0:N+1]


    cp = zeros_like(b)

    cp[0] = sum(b[1::2])

    evens = b[2::2]
    odds = b[1::2]

    cp[1:N+1-(N%2):2] = 2*cumsum(evens[-1::-1])[-1::-1]
    cp[2:N+1-((N+1)%2):2] = 2*cumsum(odds[-1::-1])[-2::-1]

    cp = cp*factor

    return cp


def chebD_semiinf(c):
    '''Finds the derivative of Chebyshev series in spectral space
    i.e. maps c_n --> d_n where c_n, d_n are Chebyshev coefficients
    of f(x) and f'(x) in the interval [0,oo)'''
    '''To be used only for the positive half-line'''
    
    N = len(c) - 1
    
    b = c*r_[0:N+1]


    cp = zeros_like(b)

    cp[0] = sum(b[1::2])

    evens = b[2::2]
    odds = b[1::2]

    cp[1:N+1-(N%2):2] = 2*cumsum(evens[-1::-1])[-1::-1]
    cp[2:N+1-((N+1)%2):2] = 2*cumsum(odds[-1::-1])[-2::-1]

    d0 = 3./4*cp[0] - cp[1]/2. + cp[2]/8.
    d1 = -cp[0] + 7./8*cp[1] - cp[2]/2. + cp[3]/8.
    d2 = cp[0]/4. - cp[1]/2. + 3./4*cp[2] - cp[3]/2. + cp[4]/8.
    d3 = cp[1]/8. - cp[2]/2. + 3./4*cp[3] - cp[4]/2. + cp[5]/8.
    
    dn = [ cp[i-2]/8. - cp[i-1]/2. + 3./4*cp[i] - cp[i+1]/2. + cp[i+2]/8.  for i in range(4,N-1)]

    dn1 = cp[N-1-2]/8. - cp[N-1-1]/2. + 3./4*cp[N-1] - cp[N-1+1]/2.
    
    dn2 = cp[N-2]/8. - cp[N-1]/2. + 3./4*cp[N]
    
    dn = r_[d0,d1,d2,d3,dn,dn1,dn2]

    return dn


def cheb2zD_semiinf(c):
    '''Finds the Chebyshev coefficients of the operator 2z df/dz when
    f has a series in Chebyshev rational functions Rn(z) = Tn((z-1)/(z+1)). Input
    is the coefficients of f.'''
    
    N = len(c) - 1
    
    b = c*r_[0:N+1]


    cp = zeros_like(b)

    cp[0] = sum(b[1::2])

    evens = b[2::2]
    odds = b[1::2]

    cp[1:N+1-(N%2):2] = 2*cumsum(evens[-1::-1])[-1::-1]
    cp[2:N+1-((N+1)%2):2] = 2*cumsum(odds[-1::-1])[-2::-1]

    d0 = -cp[2]/4. + cp[0]/2.
    d1 = cp[1]/4. - cp[3]/4.
    d2 = -cp[0]/2. + cp[2]/2. - cp[4]/4.
    
    dn = [ -cp[n-2]/4.  + cp[n]/2. - cp[n+2]/4 for n in range(3,N-1)] 
    
    dn1 = -cp[N-3]/4. + cp[N-1]/2
    dn2 = -cp[N-2]/4. + cp[N]/2
    
    dn = r_[d0,d1,d2,dn,dn1,dn2]
    
    return dn



def Intcheb(y,interval):
    '''Clenshaw-Curtis to find definite integral of function y(x) given at
    Chebyshev grid points in some interval [a,b]'''
    fact = 0.5*(interval[1]-interval[0])
    b = cheb(y)
    N = len(y) - 1
    if N%2 == 0:
        w = array([ 2./(-(2*k)**2+1) for k in r_[0:N/2+1]])
    else:
        w = array([ 2./(-(2*k)**2+1) for k in r_[0:(N-1)/2+1]])
    return dot(b[::2],w)*fact



def chebI(c,interval,x0=None,f0=None):
    if x0==None:
        x0=interval[0]

    N = len(c) - 1
    I = diag(1./(2*r_[0.5,r_[2:N+1]]),-1) -diag(1./(2*r_[1,r_[1:N]]),1)
    I[0,1] = 0
    
    factor = (interval[1]-interval[0])/2.
    
    ci = dot(I,c)*factor
    x = 2*(x0-interval[0])/(interval[1]-interval[0]) - 1
    
    if x==-1 and f0==None:
        ci[0] = -sum((-1)**r_[1:N+1]*ci[1:])
    else:
        ci[0] = f0 - clenshaw(x,ci)
    return ci



def cheb_convolve(a,b):
    '''Finds the product of two functions whose Chebyshev coefficients are 
    given by a and b. Output is the coefficiets of the product.'''

    M = len(b)
    N = len(a)
    
    if N>M:
        b = r_[b,zeros(N-M)]
        N = N - 1
    elif M>N:
        a = r_[a,zeros(M-N)]
        N = M - 1
    else:
        N = N - 1
    
    a[0] = a[0]*2.
    b[0] = b[0]*2.
        
    c0 = a[0]*b[0] + 2*dot(a[1:],b[1:])
    
    c1 = [ dot(a[0:k+1][::-1],b[0:k+1]) + dot(a[1:N-k+1],b[k+1:N+1]) + dot(a[k+1:N+1],b[1:N-k+1])  for k in range(1,N) ]

    c2 = [ dot(a[k-N:N+1][::-1],b[k-N:N+1])  for k in range(N,2*N+1)]
    
    c = r_[c0/2,c1,c2]/2.
    
    return c[:N+1]
    
def cosT(d,inverse=False):
    '''Finds the cosine transform of a given sequence'''
    b = []
    N = len(d)-1
    for n in r_[0:N+1]:
        b.append(sum(d*cos(n*r_[0:N+1]*pi/N)))
    b = array(b)
    if inverse:
        return b
    else:
        b[0] = b[0]/(N)
        b[1:] = b[1:]*2/(N)
        return b

In [4]:
def cintmatrix(N, interval=[-1,1]):
    A=np.eye(N)
    B=np.zeros((N,N))
    i=0
    for row in A:
        B[i,:]= chebI(chebI(row,interval,-1,0),interval,1,0)
        i+=1
    T=B.transpose() 
    return T

In [5]:
N = 100
I= cintmatrix(N)
16/(lam2*pi**2)

NameError: name 'lam2' is not defined

In [6]:
def loader(i,N):
    e = np.zeros(N)
    e[i] = 1
    j=np.empty((N,))
    j[::2]=1
    j[1::2]=-1
    p = chebI(chebI(e,[-1,1],-1,0),[-1,1],1,0)
    return dot(p,j)

In [16]:
N= 7
w= empty(N)
for i in range(N):
    w[i] = loader(i,1000)
w

array([-2.        ,  0.66666667,  0.66666667, -0.4       ,  0.13333333,
       -0.0952381 ,  0.05714286])

In [60]:
N=50
e = np.zeros(N)
e[1] = -1
e[0] =  1
I = np.identity(N)
B = np.zeros((N+1,N+1))
B[0:N,0:N] = I
B[0:N,-1] = e
B[-1,-1] = 2
Int = cintmatrix(N)
A = np.zeros((N+1,N+1))
A[0:N,0:N] = Int
k = np.ones(N)
w = empty(N)
for i in range(N):
    w[i] = loader(i,1000) 
A[-1,0:N] = w

In [61]:
e= reshape(e, (N,1))
shape(e)
w= reshape(w,(1,N))
shape(w)
tild = 0.5*dot(e,w)
shape(tild)
MatI= Int - tild

Method 1

In [59]:
lam1= lin.eigvals(A,B)
-4/(lam1*pi**2)

array([  1.00000000e+00    -0.j        ,   9.00000000e+00    -0.j        ,
         4.00000000e+00    -0.j        ,   2.50000000e+01    -0.j        ,
         1.60000000e+01    -0.j        ,   4.90000000e+01    -0.j        ,
         3.60000000e+01    -0.j        ,   8.10000000e+01    -0.j        ,
         6.40000000e+01    -0.j        ,   1.21000000e+02    -0.j        ,
         1.00000000e+02    -0.j        ,   1.69000000e+02    -0.j        ,
         1.44000000e+02    -0.j        ,   2.25000000e+02    -0.j        ,
         1.96000000e+02    -0.j        ,   2.89000000e+02    -0.j        ,
         2.56000000e+02    -0.j        ,   3.60999999e+02    -0.j        ,
         4.41000034e+02    -0.j        ,   3.24000002e+02    -0.j        ,
         5.28999050e+02    -0.j        ,   3.99999840e+02    -0.j        ,
         6.25015516e+02    -0.j        ,   4.84006698e+02    -0.j        ,
         7.28904305e+02    -0.j        ,   5.75847569e+02    -0.j        ,
         8.42953833e+02  

Method 2

In [62]:
lam2= lin.eigvals(MatI)
-4/((lam2*pi**2))

array([  1.00000000e+00    -0.j        ,   9.00000000e+00    -0.j        ,
         4.00000000e+00    -0.j        ,   2.50000000e+01    -0.j        ,
         4.90000000e+01    -0.j        ,   1.60000000e+01    -0.j        ,
         8.10000000e+01    -0.j        ,   3.60000000e+01    -0.j        ,
         1.21000000e+02    -0.j        ,   6.40000000e+01    -0.j        ,
         1.00000000e+02    -0.j        ,   1.69000000e+02    -0.j        ,
         1.44000000e+02    -0.j        ,   2.25000000e+02    -0.j        ,
         1.96000000e+02    -0.j        ,   2.89000000e+02    -0.j        ,
         2.56000000e+02    -0.j        ,   3.60999999e+02    -0.j        ,
         3.24000002e+02    -0.j        ,   4.41000034e+02    -0.j        ,
         3.99999840e+02    -0.j        ,   5.28999050e+02    -0.j        ,
         6.25015516e+02    -0.j        ,   4.84006698e+02    -0.j        ,
         7.28904305e+02    -0.j        ,   5.75847569e+02    -0.j        ,
         8.42953833e+02  

Method 3

In [51]:
lam3 = lin.eigvals(Mat2, Int)
-4/((lam2*pi**2))

/home/jithin/anaconda3/lib/python3.6/site-packages/scipy/linalg/decomp.py:60: RuntimeWarning: divide by zero encountered in true_divide
  w = (alphar + _I * alphai) / beta
/home/jithin/anaconda3/lib/python3.6/site-packages/scipy/linalg/decomp.py:60: RuntimeWarning: invalid value encountered in true_divide
  w = (alphar + _I * alphai) / beta


array([  1.37648159e-01-0.j,   2.14788874e+00-0.j,   6.14840133e+00-0.j,
         1.21485382e+01-0.j,   2.01485940e+01-0.j,   3.01486221e+01-0.j,
         4.21486382e+01-0.j,   5.61486484e+01-0.j,   7.21486552e+01-0.j,
         9.01486608e+01-0.j,   1.10148649e+02-0.j,   1.32148529e+02-0.j,
         1.56149722e+02-0.j,   1.82154590e+02-0.j,   2.10127103e+02-0.j,
         2.40120452e+02-0.j,   2.72867577e+02-0.j,   3.09181823e+02-0.j,
         3.50360940e+02-0.j,   4.04728245e+02-0.j,   4.85043923e+02-0.j,
         5.93275533e+02-0.j,   7.34721485e+02-0.j,   9.72240284e+02-0.j,
        -3.85865547e+16+0.j,   4.93702381e+04-0.j,   7.74716198e+03-0.j,
         3.53324827e+03-0.j,   2.20901476e+03-0.j,   1.42909328e+03-0.j])

### Mixed boundary conditons

$y''=\lambda y$

y'(-1)=0,y(1)=0

$y(x)= \int_{1}^{x}\int_{-1}^{s}y(p)dpds$


$\frac{1}{\lambda}y(x)= Iy$

In [8]:
lam3= lin.eigvals(Int)
-1/(lam3*pi**2/16)

array([  1.00000000e+00-0.j,   9.00000000e+00-0.j,   2.50000000e+01-0.j,
         4.90000000e+01-0.j,   8.10000000e+01-0.j,   1.21000000e+02-0.j,
         1.69000000e+02-0.j,   2.25000000e+02-0.j,   2.89000000e+02-0.j,
         3.61000000e+02-0.j,   4.41000000e+02-0.j,   5.29000000e+02-0.j,
         6.25000000e+02-0.j,   7.29000000e+02-0.j,   8.41000000e+02-0.j,
         9.61000000e+02-0.j,   1.08900000e+03-0.j,   1.22500000e+03-0.j,
         1.36900000e+03-0.j,   1.52100000e+03-0.j,   1.68100000e+03-0.j,
         1.84900000e+03-0.j,   2.02500000e+03-0.j,   2.20900000e+03-0.j,
         2.40100000e+03-0.j,   2.60100000e+03-0.j,   2.80900000e+03-0.j,
         3.02500000e+03-0.j,   3.24900000e+03-0.j,   3.48100000e+03-0.j,
         3.72100000e+03-0.j,   3.96900000e+03-0.j,   4.22500000e+03-0.j,
         4.48900000e+03-0.j,   4.76100000e+03-0.j,   5.04100000e+03-0.j,
         5.32900000e+03-0.j,   5.62500000e+03-0.j,   5.92900000e+03-0.j,
         6.24100000e+03-0.j,   6.56100000e+03-0.j, 

In [9]:
import scipy.linalg as p

In [10]:
lam= p.eigvals(A,B)

In [11]:
lam

array([ -2.94435274e+00+0.j,  -1.88689818e-01+0.j,  -6.59170917e-02+0.j,
        -3.33607821e-02+0.j,  -2.01147899e-02+0.j,  -1.34428941e-02+0.j,
        -9.61560685e-03+0.j,  -7.21806751e-03+0.j,  -5.61735675e-03+0.j,
        -4.49573777e-03+0.j,  -3.67943398e-03+0.j,  -3.06688480e-03+0.j,
        -2.59550555e-03+0.j,  -2.22502166e-03+0.j,  -1.92856197e-03+0.j,
        -1.68764096e-03+0.j,  -1.48920342e-03+0.j,  -1.32381673e-03+0.j,
        -1.18452815e-03+0.j,  -1.06612166e-03+0.j,  -9.64622190e-04+0.j,
        -8.76957474e-04+0.j,  -8.00722701e-04+0.j,  -7.34013776e-04+0.j,
        -6.75307221e-04+0.j,  -6.23372391e-04+0.j,  -5.77206435e-04+0.j,
        -5.35985511e-04+0.j,  -4.99027760e-04+0.j,  -4.65764927e-04+0.j,
        -4.35720380e-04+0.j,  -4.08491937e-04+0.j,  -3.83738335e-04+0.j,
        -3.61168482e-04+0.j,  -3.40532861e-04+0.j,  -3.21616601e-04+0.j,
        -3.04233860e-04+0.j,  -2.88223245e-04+0.j,  -2.73444048e-04+0.j,
        -2.59773149e-04+0.j,  -2.47102437e-04+0.j, 

In [17]:
for i in range(1,20):
    l=1/(i)**2
    print(4*l/np.pi**2)

0.4052847345693511
0.10132118364233778
0.04503163717437234
0.025330295910584444
0.016211389382774045
0.011257909293593086
0.008271117032027573
0.006332573977646111
0.005003515241596927
0.004052847345693511
0.003349460616275629
0.0028144773233982714
0.002398134524079001
0.0020677792580068933
0.0018012654869748938
0.0015831434944115277
0.001402369323769381
0.0012508788103992319
0.0011226723949289503


In [18]:
1/lam

array([ -1.35853288e+00 -0.00000000e+00j,
        -2.46740110e+00 -0.00000000e+00j,
        -2.11988121e+01 -0.00000000e+00j,
        -2.22066099e+01 -0.00000000e+00j,
        -6.06822889e+01 -0.00000000e+00j,
        -6.16850275e+01 -0.00000000e+00j,
        -1.19901266e+02 -0.00000000e+00j,
        -1.20902654e+02 -0.00000000e+00j,
        -1.98858652e+02 -0.00000000e+00j,
        -1.99859489e+02 -0.00000000e+00j,
        -2.97554973e+02 -0.00000000e+00j,
        -2.98555533e+02 -0.00000000e+00j,
        -4.15990385e+02 -0.00000000e+00j,
        -4.16990786e+02 -0.00000000e+00j,
        -5.54164947e+02 -0.00000000e+00j,
        -5.55165248e+02 -0.00000000e+00j,
        -7.12078684e+02 -0.00000000e+00j,
        -7.13078918e+02 -0.00000000e+00j,
        -8.89731610e+02 -0.00000000e+00j,
        -8.90731797e+02 -0.00000000e+00j,
        -1.08712373e+03 -0.00000000e+00j,
        -1.08812389e+03 -0.00000000e+00j,
        -1.30425505e+03 -0.00000000e+00j,
        -1.30525518e+03 -0.0000000